# Pragmas, modules and cabal

## Outline

* Prelude module

* Using pragmas

* Working with base modules 

* Creating our own modules

* Using cabal

In this lesson, we will learn how you can work with Haskell modules, pragmas and the cabal tool.

## Prelude module

When working in GHCi some function are available by default as for instance `head`, `sum` and `length`. This means nothing has to be imported or installed in Haskell for them to work. It's because those function are part of the standard Haskell module called **Prelude** that is imported by default. You can find a list of all the functions contained in **Prelude** on the Hackage site about which we will talk in the next lesson: https://hackage.haskell.org/package/base-4.17.0.0/docs/Prelude.html 

One of the more used **Prelude** function are:

- `head` (gives you the first element of the list)
- `tail` (gives you the elements of the list except the first one)
- `sum` (sums elements of a list)
- `lenght` (gives the lenght of a list)
- `print` (prints a varibale to the terminal)

Some others we have also introduced in lesson 6. On the hackage link of the prelude module provided above, you will also find the type signatures of all the functions.

## Using pragmas

Pragmas or language extensions are a way to add some functionality to your Haskell code that is not there by default. Pragmas should be placed at the top of each source file. In lesson 11 you learned about bytestrings and how to display them. There is a pragma called **OverloadedStrings** which when added to your file allows you to write bytestrings as normal strings without having to use the function pack. You can see how this is done in the example below.   

In [ ]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString as BS

bytestring :: BS.ByteString
bytestring = "1"  -- would throw error without the pragma

main :: IO ()
main = do
  print bytestring

main

There are many other pragmas that you can add to your code. Some of them are:

- **NoImplicitPrelude**: In Plutus we prefer to use a custom Prelude that uses strict functions by default. This language extension allows us not use the default Prelude.
- **TemplateHaskell**: Provides tools for Haskell metaprogramming which means that the code generates other code. Is also used in Plutus.
- **ViewPatterns**: Allows for more-sophisticated pattern matching.

## Working with base modules

Some functions in Haskell need to be imported with modules that are available beside the standard **Prelude** module. Haskell defines many modules for your convinience. For instance let's say you want to run a haskell file from the command line by giving it some input parameters and then read them from the program. You can do this with the function `getArgs` that is part of the `System.Environment` module.

In [ ]:
import System.Environment ( getArgs )

main :: IO ()
main = do
  args <- getArgs
  print $ args !! 0

Because we wrote the `getArgs` function in the import statement only this function was imported from the module. If we would instead written `import System.Environment` all function from the System.Environment module would have been imported.

Now you might be asking yourself how you know which module to import and which function to use. The best answer is always google what you want to achive and once you find a module or function that fits your needs you can use Hackage and Hoogle to learn more about them. We will discuss those web services in the next lesson. For our example of the **getArgs** function you could for instance write into google: `haskell get command line arguments`.

### Base modules Data.Char, Data.Tuple and Data.Array

When reading about these modules try to import them in GHCi and play with some of the functions. You can always check the type signature of a function with `:t <function_name>`.

The **Data.Char** module defines functions that deal with characters. Some of the often used functions are:

- `isDigit` (checks if a character is a digit)
- `isPunctuation` (checks if a character is a punctuation)
- `toUpper` and `toLower` (converts a lower case character to upper or vice versa)
- `ord` and `chr` (converts a character to ascii code number or vice versa)

In [ ]:
import Data.Char

isDigit '1'
isPunctuation '.'
toUpper 'a'
ord 'a'

The **Data.Tuple** module defines functions that deal with tuples. Some of the often used functions are:

- `fst` (gives you the first element of a tuple)
- `snd` (gives you the second element of a tuple)
- `swap` (swaps elements in a 2-element tuple)

In [ ]:
import Data.Tuple

fst (1,2)
snd (1,2)
swap (1,2)

The **Data.Array** module defines functions that deal with arrays. Some of the often used functions are:

- `listArray` (construct an array from a pair of bounds and a list of values in index order)
- `(!)` (access the value at the given index in an array)
- `indices` (the list of indices of an array in ascending order)
- `elems` (the list of elements of an array in index order)

In [ ]:
import Data.Array

myArray = listArray (1,3) [4,5,6]
-- array (1,3) [(1,4),(2,5),(3,6)]

myArray ! 1 -- 4
indices myArray -- [1,2,3]
elems myArray -- [4,5,6]

## Creating your own modules

Modules are just plain haskell files that define some functions. You can create a module on your own. Lets say we want another version of the Prelude function `sum` that by default returns for an empty list the value 0. Instead we want an error message to be displayed for the empty list. 

First we create our module file that we call sum.hs and write a module statement in the begining of the file. The name of the module should be same as the name of the file, just first letter uppercase. Then we define our own parameterized type `Check` that can be used for displaying an error message. We will learn in depth about handling errors in the chapter **Maybe and Either**. After that we define our `sum` function.

In [ ]:
module Sum where

data Check a b = Error a | Result b

sum :: Num a => [a] -> Check String a
sum [] = Error "List is empty"
sum xs = Result $ Prelude.sum xs

Notice that in the definition of our `sum` function we use the prelude version of `sum` that we acces by `Prelude.sum`. Now if we were in another haskell file and wanted to import our Sum module we would have to do this with a qualified import to avoid name colision of both `sum` functions.

```haskell
import qualified Sum as SumModule

Prelude.sum []       -- 0
Prelude.sum [1..3]   -- 6

SumModule.sum []     -- Error "List is empty" 
SumModule.sum [1..3] -- Result 6
```

If you would not want to rename the Sum module you could simply write `import qualified Sum` and then you would access the function with `Sum.sum`. If your user defined sum function name however would not match any default function name from Prelude, you could use a simple import statement as `import Sum`. This would work if our function name would be e.g. `sum1`. Then you could use the function names directly. 

```haskell
import Sum

sum []      -- 0
sum [1..3]  -- 6

sum1 []     -- Error "List is empty" 
sum1 [1..3] -- Result 6
```

**NOTE**: When compiling a haskell file with GHC or loading it in GHCi all imported modules will compile automatically given that the files that specify them are in the same directory as the file you are directly compiling or loading.

## Using cabal

### Instalation of packages with cabal

It is important to state that some modules can be directly imported into Haskell code because the packages that contains them get shiped with the installation of GHC and GHCi. The **base** and **container** packages are just two of them. But there are also other packages which need to be installed in order to be able to use their modules in Haskell code. One of them is for instance the **aeson** package that provides modules for processing JSON data. The packages are hosted on the Hackage site (*hackage.haskell.org*) about which we will talk in the next lesson. If you look at the **aeson** Hackage page you see that it contains the **Data.Aeson** module

In order to check weather a module can be imported in Haskell simply start GHCi and type `import <module_name>`. If the package containing the module is not installed you will get an error. You can also use the TAB button for auto-completion. For instnce type `import Data.A` and hit TAB. You wil get a list of all modules that start with `Data.A`. In the default installation of Haskell you will not see the `Data.Aeson` module so you know you have to install it. For installation of packages the **cabal** command is used which stands for *Common Architecture for Building Applications and Libraries*.

To install a package from Hackage you can use the command:
```
cabal install <package_name>
```

It could happen that cabal will complain if the package is in form of a library. In this case use the command:
```
cabal install --lib <package_name>
```

If you download a tar.gz file for instance, you can install the package locally with the command:
```
cabal install ./<package_name>.tar.gz
```

### Building a project with cabal

You can also use cabal to build Haskell projects. Ussually when your project contains a Main.hs and many .hs files for libraries it is better to use cabal then directly compiling the Main.hs file. To create a new project in cabal you can use the following commands:

Creates a simple project:
```
cabal init
```

Creates a project by asking you multiple questions where you can choose from a set of parameters. In the begining you should say no for simple project:
```
cabal init --interactive
```

The name of your project for the first command will be set to the name of the folder on which you created the project. An app/ directory will be created with a Main.hs file in which your main function will reside and a .cabal file will be created that holds all the information for building the project. Lets have a look at the file contents: 

```
cabal-version:      2.4
name:               test
version:            0.1.0.0

-- A short (one-line) description of the package.
-- synopsis:

-- A longer description of the package.
-- description:

-- A URL where users can report bugs.
-- bug-reports:

-- The license under which the package is released.
-- license:
author:             Luka Kurnjek
maintainer:         luka.kurnjek@iohk.io

-- A copyright notice.
-- copyright:
-- category:
extra-source-files: CHANGELOG.md

executable test
    main-is:          Main.hs

    -- Modules included in this executable, other than Main.
    -- other-modules:

    -- LANGUAGE extensions used by modules in this package.
    -- other-extensions:
    build-depends:    base ^>=4.14.3.0
    hs-source-dirs:   app
    default-language: Haskell2010
```

The fields in the begining are pretty much self-explanatory. You can also remove the comments for some of the fields that were also added to the file. In the executable section information for building the Main.hs of the project are stated. The build-depends flag states any libraries that are needed to build the project. The base library is always added by default. As you can see cabal supports equality and inequality operators for comparing versions. The carrot operator ^ is used to treat ^>= x.y.z as identical to >= x.y.z && < x.(y + 1). So in our case it would be `base >= 4.14.3.0 && < 4.15`. Also the directory of the Main.hs file is speciefie and the default language which is the newest at current time Haskell2010.

In order to build your project you run the command:
```
cabal build
```

In order to run your project you run the command:
```
cabal exec <project_name>
```

You can do both actions also with one command:
```
cabal run
```

If you use the interactive method for create the project you have the option to add libraries and test cases. If you do so, the cabal tool will generate the src/ (or lib/) and test/ directories and the .cabal file will have also configurations for library and test-suite.

## Recap

In this lesson, we have discussed:

- the Prelude module and its functions
- language pragmas and how to use them
- how to work with base modules
- how to create your own modules
- using cabal for installing packages and building projects